# Introducing `sb4dfritzlib`

I invested some time to understand how to communicate with AVM FRITZ!Box routers using the [official APIs](https://fritz.com/service/schnittstellen/). As a result, I have written my own library `sb4dfritzlib` that will eventually remove the dependency of `sb4dfritz` from the `fritzconnection` library. Not that there's anything wrong with the latter, I just wanted to know how to do it myself so that I know exactly what's going on.

Another motivating factor was the faint hope to get more accurate "real time" power measurements from energy sensors, or at least a better understanding what causes the limitations that I encountered. Recall that `fritzconnection` returns timestamps with second precision, leaving it unclear whether they were obtained by truncating or rounding. While I had little doubt that AVM is the root of this imprecision, I couldn't rule out the data processing `fritzconnection` as a cause. I can now definitively say that `fritzconnection` simply converts the XML data returned by the FRITZ!Box to appropriate Python data types. 

Anyway, here's a summary of what I did:
* I implemented the [SOAP](https://en.wikipedia.org/wiki/SOAP)-based TR-064 interface (see `sb4dfritzlib.connection`).
* I implemented the simpler AHA-HTTP interface and the required login procedure (see `sb4dfritzlib.connection`).
* I implemented various utilities for data conversion (see `sb4dfritzlib.utilities`).
* The core functionality is implemented in `sb4dfritzlib.homeauto` which provides classes `HomeAutoSystem` and `HomeAutoDevice` for object-based interaction with home automation devices.

## Comparing `sb4dfritzlib` and `fritzconnection`



In [5]:
import importlib
import sys
import json 

import fritzconnection
import sb4dfritzlib
_ = importlib.reload(sys.modules['sb4dfritzlib'])

from sb4dfritzlib.connection import FritzUser
from sb4dfritzlib.homeauto import HomeAutoSystem

CONFIG_FILE = "..\\..\\_private_files\\sb4dfritz_secrets.ini"
with open(CONFIG_FILE,"r") as file:
    CONFIG = json.load(file)
# extract login data
USER = CONFIG['login']['user']
PWD = CONFIG['login']['pwd']
IP = CONFIG['login']['ip']


Here's how to connect to the home automation system using `sb4dfritzlib`:

In [52]:
# Create FritzUser from credentials
sb4d_user = FritzUser(USER, PWD, IP)
# Connect to home automation system
sb4d_home_auto_system = HomeAutoSystem(sb4d_user)
# List registered devices
print("Registered home automation devices:")
for device in sb4d_home_auto_system.devices:
    print(" -", device)

Registered home automation devices:
 - l'angolo del caffè (AVM FRITZ!Smart Energy 200)
 - radiatore nel soggiorno (AVM FRITZ!Smart Thermo 301)
 - radiatore musicale (AVM FRITZ!Smart Thermo 301)
 - radiatore nella cucina (AVM FRITZ!Smart Thermo 301)
 - TV etc (AVM FRITZ!Smart Energy 200)


Under the hood, the instantiation of `HomeAutoSystem` first log in to obtains a valied session id, then asks for a list of connected devices, and creats a `HomeAutoDevice` object for each device. 

In my case, the first device is a smart plug which functions as an on/off switch for my espresso machine. Let's check it's current power consumption:

In [53]:
# Give coffe machine a dedicated variable
sb4d_caffe = sb4d_home_auto_system.devices[0]
# Get latest power measurement together with timing information
power_record = sb4d_caffe.get_latest_power_record()
print("Current power consumption and timing information:")
for key, val in power_record.items():
    print(f" - {key:15s} {val}")

Current power consumption and timing information:
 - power           3.29
 - datatime        2025-08-20 13:32:59
 - starttime       2025-08-20 13:34:53.875762
 - endtime         2025-08-20 13:34:54.717654
 - duration        0.841892
 - latency         115.717654
 - offset          -114.875762


As you can guess, the smart plug reported 0.0 W as `power` at the time `datatime`. 
The parameters `starttime` and `endtime` record the times that the network request to obtain the data was sent and received, respectively.
Lastly, `duration` is the total request time in seconds while `latency` and `offset` compare the `endtime` and `starttime` to `datatime`, respectively.
The large values are typical if no request has been made recently. 

At this point, it should not go unnotices that a latency of over 50 seconds, meaning that the measurement was over 50 seconds ago, can hardly be considered "real time". 

Let's make a few more requests to see what happens over time.

In [48]:
out_template = """{:4s} Request: {:s} | Power: {:0.1f} W | Timestamp: {:s} | Duration: {:0.2f} | Latency: {:0.2f}"""
for k in range(25):
    data = sb4d_caffe.get_latest_power_record()
    output = out_template.format(
        f"({k+1})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)


(1)  Request: 12:05:01.674186 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.77 | Latency: 16.45
(2)  Request: 12:05:02.445643 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.74 | Latency: 17.19
(3)  Request: 12:05:03.186501 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.76 | Latency: 17.94
(4)  Request: 12:05:03.943442 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.82 | Latency: 18.76
(5)  Request: 12:05:04.760071 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.76 | Latency: 19.52
(6)  Request: 12:05:05.523466 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.75 | Latency: 20.28
(7)  Request: 12:05:06.278217 | Power: 0.0 W | Timestamp: 12:04:46 | Duration: 0.75 | Latency: 21.03
(8)  Request: 12:05:07.032048 | Power: 0.0 W | Timestamp: 12:05:06 | Duration: 0.82 | Latency: 1.85
(9)  Request: 12:05:07.847494 | Power: 0.0 W | Timestamp: 12:05:06 | Duration: 1.10 | Latency: 2.94
(10) Request: 12:05:08.945175 | Power: 0.0 W | Timestamp: 12:05:06 | Duration: 0.76 | Latency

From the above results, we can take away a few thing:

* Request normally take around 0.75 to 0.8 seconds, occasionally a little longer.
* The timestamp jumps 10 second interval. This means that power is measured only every 10 seconds. It also means that the **data requests *do not* trigger a measurements!** Measurements are taken periodically at times unrelated to the requests.
* On repeated requests, the latency does not exceed 10 seconds by much. This indicates some undocumented "sleep mode" which reduces the update frequency in longer time stretches without data requests.

If you're interested in real time information, this is bad news. A random request will give you a power value recorded some time in the last ~12.5 seconds, if you're lucky. Again, I would not consider this as "real time". After a little thought, you should realize that,  given the network latency (reflected in the request duration) and the second presicion in the timestamp, it's impossible to determine the time of measurement precisely from the provided data. There is always going to be some latency, and in bad cases you cannot expect to do better than ~2 seconds of latency. 

My personal motivation was to switch off my coffee machine only when it's idle (indicated by a power consumption below 5 W). The problem is that the machine has irregular heating cycles, even if it's not actively being used. 

**Problem:** How to time the data requests to guarantee minimal possible latency?

